<a href="https://colab.research.google.com/github/XiaoHuang0803/ColabNotebooks/blob/main/Agents_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install pyautogen

In [3]:
from google.colab import userdata
OPEN_AI_ACCESS_KEY = userdata.get('openAIAccessKey')

In [19]:
from autogen import config_list_from_json, ConversableAgent, UserProxyAgent, AssistantAgent

In [20]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

In [21]:
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

In [22]:
user_proxy = UserProxyAgent(
    "user",
    code_execution_config={
        "work_dir": "working",
        "use_docker": False,
        "last_n_messages": 1,
    },
    human_input_mode = "ALWAYS",
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE")
)


In [24]:
engineer = AssistantAgent(
    name="Engineer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional Python engineer, known for your expertise in software development.
    You use your skills to create software applications, tools and games that are both functional and efficient.
    You preference is to write clean, well-structured code that is easy to read and maintain
    """ ,
)

In [25]:
critic = AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a code reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize code content for any harmful or substandard elements.
    You ensure that the code is secure, efficient, and adhere to best practices.
    You will identify any issues or areas for improvement in the code and output them as a list.
    """
)

In [26]:
# user_proxy.initiate_chat(assistant, message="What should I do if someone in my team who does not perform as good as me got promoted first?")

In [27]:
def review_code(recipient, messages, sender, config):
    return f"""
    Review and critique the following code.

    {recipient.chat_messages_for_summary(sender)[-1]["content"]}
    """

In [28]:
user_proxy.register_nested_chats(
    [
        {
            "recipient": critic,
            "message": review_code,
            "summary_method": "last_msg",
            "max_turn": 1
        }
    ],
    trigger=engineer,
)

In [29]:
task = """Write a snake game using Python."""
res = user_proxy.initiate_chat(
    recipient=engineer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

user (to Engineer):

Write a snake game using Python.

--------------------------------------------------------------------------------
Engineer (to user):

Sure, here's a quick and simple implementation of a console-based snake game in Python. For visual elements, I used the `curses` built-in module. 

```python
import random
import curses

s = curses.initscr()
curses.curs_set(0)
sh, sw = s.getmaxyx()
w = curses.newwin(sh, sw, 0, 0)
w.keypad(1)
w.timeout(100)

snk_x = sw//4
snk_y = sh//2
snake = [
    [snk_y, snk_x],
    [snk_y, snk_x-1],
    [snk_y, snk_x-2]
]

food = [sh//2, sw//2]
w.addch(int(food[0]), int(food[1]), curses.ACS_PI)

key = curses.KEY_RIGHT

while True:
    next_key = w.getch()
    key = key if next_key == -1 else next_key

    if snake[0][0] in [0, sh] or \
        snake[0][1]  in [0, sw] or \
        snake[0] in snake[1:]:
        curses.endwin()
        quit()

    new_head = [snake[0][0], snake[0][1]]

    if key == curses.KEY_DOWN:
        new_head[0] += 1
    if